In [1]:
pwd

'/home/kate/minecraft_corpus/stats/stats_pickles'

In [2]:
map_relations = {'Comment': 0, 'Contrast': 1, 'Correction': 2, 'Question-answer_pair': 3, 'Parallel': 4, 'Acknowledgement': 5,
            'Elaboration': 6, 'Clarification_question': 7, 'Conditional': 8, 'Continuation': 9, 'Result': 10, 'Explanation': 11,
            'Q-Elab': 12, 'Alternation': 13, 'Narration': 14, 'Confirmation_question': 15, 'Break': 16, 'Sequence' : 17, 'Background': 18}

In [3]:
reverse_map = {0: 'Comment', 1:'Contrast', 2:'Correction', 3:'QAP', 4:'Parallel', 5:'Acknowledgement',
            6:'Elaboration', 7:'Clarification_question', 8:'Conditional', 9:'Continuation', 10:'Result', 11:'Explanation',
            12:'Q-Elab', 13:'Alternation', 14:'Narration', 15:'Conf-Q', 16: 'Break', 17:'Sequence', 18:'Background', -1:'Break'}

In [4]:
amend_map_relations = {'Comment': 0, 'Contrast': 1, 'Correction': 2, 'Question-answer_pair': 3,
                       'Acknowledgement': 5, 'Elaboration': 6, 'Clair_elab': 4, 'Conditional': 8, 
                       'Continuation': 9, 'Result': 10, 'Explanation': 11, 'Alternation': 13, 
                       'Narration': 14, 'Confirmation_question': 15, 'Break': 16, 'Sequence' : 17}

In [5]:
amend_reverse_map = {0: 'Comment', 1:'Contrast', 2:'Correction', 3:'QAP', 4:'Clair_elab', 5:'Acknowledgement',
            6:'Elaboration', 8:'Conditional', 9:'Continuation', 10:'Result', 11:'Explanation',
           13:'Alternation', 14:'Narration', 15:'Conf-Q', 16: 'Break', 17:'Sequence'}

In [6]:
import pickle
import json
from collections import Counter, defaultdict 
from sklearn.metrics import precision_recall_fscore_support

In [7]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix

In [8]:
home = %pwd

In [9]:
dev_path = home + '/DEV_32_bert.json'

In [10]:
dev_path

'/home/kate/minecraft_corpus/stats/stats_pickles/DEV_32_bert.json'

In [32]:
with open(dev_path, 'r') as jf:
    dev_32 = json.load(jf)

In [33]:
len(dev_32)

32

##step one 
adds info to edus that is necessary for more second pass (Narration) processing.
info: global turn, architect edu index in turn, 1/0 result incoming, edu type
returns only narration relations. 

In [13]:
def is_nl(edu):
    """
    if every word in alphanumeric
    """
    nl = 1
    words = edu.split(' ')
    # print(words)
    for word in [w for w in words if w != '']:
        if not contains_number(word):
            nl = 0
            break
    return nl

def contains_number(string):
    return any(char.isdigit() for char in string)

In [14]:
dev_32[0]['edus'][0]

{'text': 'Mission has started.', 'speaker': 'Builder'}

In [41]:
for game in dev_32_bert:
# for game in dev_32:
#     rels = game['relations']
    rels = game['pred_rels']
    edus = game['edus']
    new_rels = []

    #add turn index for arch and global turn info to all edus
    ind_cnt = 0
    global_cnt = 0
    last_speaker = None
    global_index = 0
    for edu in edus:
        edu['global_index'] = global_index
        global_index += 1
        speaker = edu['speaker']
        if speaker == last_speaker:
            edu['turn'] = global_cnt
        else:
            last_speaker = speaker
            global_cnt += 1
            edu['turn'] = global_cnt
        if speaker == 'Architect':
            edu['turn_ind'] = ind_cnt
            ind_cnt += 1
            edu['type'] = 0
        elif speaker == 'Builder':
            ind_cnt = 0
            #also add type info
            if is_nl(edu['text']):
                edu['type'] = 1 
            else:
                edu['type'] = 0
        #add field for incoming result information
        edu['res'] = 0
        edu['ack'] = 0
        edu['nar_start'] = 0

    #add incoming Result information
    for rel in rels:
        ind = rel['y']
        if rel['type'] == 'Result':
            edus[ind]['res'] = 1
        if rel['type'] == 'Acknowledgement':
            edus[ind]['ack'] = 1
        #add first continuation!!!
        if rel['x'] == 0 and rel['type'] == 'Continuation':
            edus[ind]['nar_start'] = 1

        #keep only specified type of relations
#         if rel['type'] == 'Narration':
#             new_rels.append(rel)         
#     game['relations'] = new_rels

In [18]:
#dev_32[0]['edus'][4:8]

In [19]:
dev_32[0]['relations'][0]

{'x': 1, 'y': 10, 'type': 'Narration'}

In [ ]:
# def get_state(edu):
#     """
#     [asdfsdf]
#     """
#     state = [0,0,0]
#     if edu['speaker'] == 'Architect':
#         state[0] = 1
        
#     return state

In [42]:
predictions = []
# for game in dev_32:
for game in dev_32_bert:
    if game['id'] not in ['C28-B29-A34', 'C151-B44-A42']:
        narration_guesses = []
        actions_happening = 0
        actions_happened = 0
        narr = [0,0]
        try:
            start = [edu['global_index'] for edu in game['edus'] if edu['nar_start'] == 1][0]
        except IndexError:
            print('no start')
            pass
        narr[0] = start
        for edu in game['edus'][start+1:]:
    #         print(edu['type'],actions_happening, actions_happened)
            if edu['type'] == 1 and actions_happening == 0: #if builder is moving
                actions_happening = 1
            if edu['type'] == 0  and actions_happening == 1: #if someone is talking now
                actions_happened = 1
            if edu['speaker'] == 'Architect' and edu['res'] == 1 and actions_happened == 1:
                narr[1] = edu['global_index']

                narration_guesses.append(tuple(narr))
                narr = [0,0]
                narr[0] = edu['global_index']
                actions_happening = 0
                actions_happened = 1
        predictions.append(narration_guesses)


no start
no start
no start
no start
no start
no start
no start


In [43]:
len(predictions)

30

In [44]:
predictions[0]

[(1, 10),
 (10, 18),
 (18, 27),
 (27, 29),
 (29, 33),
 (33, 39),
 (39, 42),
 (42, 48),
 (48, 50)]

In [23]:
#format gold
gold = []
for game in dev_32:
    narrs = []
    if game['id'] not in ['C28-B29-A34', 'C151-B44-A42']:
        print(game['id'])
        for rel in game['relations']:
            narrs.append((rel['x'], rel['y']))
        gold.append(narrs)
        

C54-B13-A30
C28-B21-A15
C142-B44-A42
C6-B12-A26
C89-B34-A31
C28-B1-A4
C148-B42-A4
C150-B54-A1
C89-B27-A25
C151-B4-A40
C150-B29-A34
C148-B54-A1
C142-B36-A35
C145-B54-A1
C54-B1-A4
C151-B29-A1
C6-B20-A10
C156-B1-A4
C89-B35-A44
C6-B1-A3
C145-B42-A4
C54-B19-A36
C151-B36-A38
C142-B53-A15
C28-B13-A30
C148-B36-A25
C54-B34-A38
C145-B35-A15
C145-B16-A26
C150-B42-A4


In [24]:
len(gold)

30

In [25]:
gold[0]

[(1, 10), (10, 27), (27, 29), (29, 33), (48, 50), (33, 39), (39, 42), (42, 48)]

In [45]:
#now rearrange
gold_narrations = []
pred_narrations = []

for i, game in enumerate(gold):
    preds = predictions[i]
    for rel in game:
        if rel in preds:
            gold_narrations.append(1)
            pred_narrations.append(1)
        if rel not in preds:
            gold_narrations.append(1)
            pred_narrations.append(0)
    for rel in preds:
        if rel not in game:
            gold_narrations.append(0)
            pred_narrations.append(1)
            

In [27]:
len(gold_narrations), len(pred_narrations)

(308, 308)

In [28]:
scores = precision_recall_fscore_support(gold_narrations, pred_narrations, average='binary')

In [29]:
scores

(0.9432624113475178, 0.910958904109589, 0.926829268292683, None)

In [ ]:
#####

In [46]:
scores = precision_recall_fscore_support(gold_narrations, pred_narrations, average='binary')

In [47]:
scores

(0.7252747252747253, 0.678082191780822, 0.7008849557522124, None)

## look at multitask output

In [30]:
#try for multi
#make new dev file
multi_preds = home + '/multi_pred_dev.pkl'
fi = open(multi_preds, 'rb')
# dump information to that file
multi = pickle.load(fi)
# close the file
fi.close()

In [31]:
multi[0]

[0, 0, 1, 9, 9]

In [34]:
dev_32_bert = dev_32

In [35]:
for i in range(32):
    rels = [m for m in multi if m[0] == i]
    new_rels = []
    for rel in rels:
        new = {}
        new['x'] = rel[1]
        new['y'] = rel[2]
        new['type'] = reverse_map[rel[4]]
        new_rels.append(new)
    dev_32_bert[i]['pred_rels'] = new_rels
        

In [36]:
dev_32_bert[0].keys()

dict_keys(['id', 'edus', 'relations', 'pred_rels'])

In [37]:
dev_32_bert[0]['relations'][0]

{'x': 0, 'y': 1, 'type': 'Continuation'}

In [38]:
dev_32_bert[0]['pred_rels'][0]

{'x': 0, 'y': 1, 'type': 'Continuation'}

??

In [ ]:
all_gold_corr = []
tp_corr = []
fp_corr = []
fn_corr = []
# nl_l_gold = []
for game in dev_32_bert:
    pred_corr = [(rel['x'], rel['y']) for rel in game['pred_rels'] if rel['type'] == 'Correction']
    gold_corr = [(rel['x'], rel['y']) for rel in game['relations'] if rel['type'] == 'Correction']
    edus = game['edus']
    game_id = game['id']
    for p in pred_corr:
        source = edus[p[0]]['speaker'] + ' : ' + edus[p[0]]['text']
        target = edus[p[1]]['speaker'] + ' : ' + edus[p[1]]['text']
        if edus[p[0]]['speaker'] == 'Builder':
            source_type = is_nl(edus[p[0]]['text'])
        else:
            source_type = 0
        if edus[p[1]]['speaker'] == 'Builder':
            target_type = is_nl(edus[p[1]]['text'])
        else:
            target_type = 0
        if p in gold_corr:
            #tp
            tp_corr.append((source, target, source_type, target_type, p[1]-p[0], game_id))
        else:
            #fp
            fp_corr.append((source, target, source_type, target_type, p[1]-p[0], game_id))
    for p in gold_corr:
        source = edus[p[0]]['speaker'] + ' : ' + edus[p[0]]['text']
        target = edus[p[1]]['speaker'] + ' : ' + edus[p[1]]['text']
        if edus[p[0]]['speaker'] == 'Builder':
            source_type = is_nl(edus[p[0]]['text'])
        else:
            source_type = 0
        if edus[p[1]]['speaker'] == 'Builder':
            target_type = is_nl(edus[p[1]]['text'])
        else:
            target_type = 0
        all_gold_corr.append([source, target, source_type, target_type, p[1]-p[0], game_id])
        if p not in pred_corr:
            fn_corr.append((source, target, source_type, target_type, p[1]-p[0], game_id))
#     #NL-L
#     for p in gold_corr:
#         source = edus[p[0]]['speaker'] + ' : ' + edus[p[0]]['text']
#         target = edus[p[1]]['speaker'] + ' : ' + edus[p[1]]['text']
#         source_type = is_nl(edus[p[0]]['text'])
#         target_type = is_nl(edus[p[1]]['text'])
#         if source_type == 1 and target_type == 0:
#             nl_l_gold.append((source, target, p[1]-p[0], game_id))
        

In [ ]:
len(all_gold_corr)

In [ ]:
len(pred_corr)

In [ ]:
len([g for g in fp_corr if g[2] == 1 and g[3] == 0])

In [ ]:
nl_to_nl_gold = [g for g in all_gold_corr if g[2] == 1 and g[3] == 1]

In [ ]:
len(nl_to_nl_gold)

In [ ]:
nl_to_nl_gold[0]

## see how many NL-NL corrections feature some undoing of a block in previous turns

In [ ]:
#measure of target undoing source
def undone(source, target):
    source_text = source.split(':')[1].split()
    target_text = target.split(':')[1].split()
    source_ones = []
    source_zeros = []
    target_ones = []
    target_zeros = []
    for s in source_text:
        if s[0] == '1':
            source_ones.append(s[1:])
        else:
            source_zeros.append(s[1:])
    for t in target_text:
        if t[0] == '1':
            target_ones.append(t[1:])
        else:
            target_zeros.append(t[1:])
#     print(source_ones)
#     print(source_zeros)
#     print(target_ones)
#     print(target_zeros)
#     total = len(source_text)
    total = len(target_text) #try with target text as a denominator
    count = 0
    for e in source_ones: 
        if e in target_zeros:
            count += 1
    for e in source_zeros:
        if e in target_ones:
            count += 1
    #count is number of target moves that undo source moves
#     print(count)
#     print(round(count/total, 2))
    return round(count/total, 2)
    
        
    

In [ ]:
undone_moves = [undone(rel[0], rel[1]) for rel in nl_to_nl_gold]

In [ ]:
Counter(undone_moves) #what percentage of the target moves are changes to source moves

In [ ]:
Counter(undone_moves) #what percentage of the source moves are changed by the target

In [ ]:
len(tp_corr), len(fp_corr), len(fn_corr)

In [ ]:
#tp NL-L
len([g for g in tp_corr if g[2] == 1 and g[3] == 0])

In [ ]:
[g for g in tp_corr if g[2] == 1 and g[3] == 0]

In [ ]:
[g for g in fn_corr if g[2] == 1 and g[3] == 0]


In [ ]:
# len([g for g in fp_corr if g[2] == 1 and g[3] == 0])
len([g for g in fp_corr])

In [ ]:
for t in fp_corr:
    print(t)
    print('---------')

In [ ]:
# for t in fn_corr:
#     print(t)
#     print('---------')

In [ ]:
# for t in fp_corr:
#     print(t)
#     print('---------')

##step 2 Takes output from second_pass.py and outputs a list of candidates, labels and raw text
for bert linear that considers only arch cans that have a non linguistic move between them. 

In [ ]:
def get_turns(edus):
    """takes a list of edus from one game and returns
    a dict of architect turns that will require skipping when forming candidates
    """
    turns = []
    last = 0
    for g in edus:
        if g['speaker'] == 'Architect':
            if g['turn'] != last:
                turns.append(g['turn'])
                last = g['turn']
        elif g['type'] == 0:
            turns.append('NL')
    # #NB:  gives us a list like this:
    # # [2, 4, 'NL', 6, 'NL', 8, 'NL', 10, 12, 13, 14, 'NL', 15, 'NL', 16, 'NL', 18, 'NL']
    # #will create a dict below like so: {2: [4], 10: [12, 13, 14], 12: [13, 14], 13: [14]}

    # print(turns)
    turn_dict = defaultdict(list)
    for i, t in enumerate(turns):
        if t != 'NL':
            n = 1
            try:
                while turns[i+n] != 'NL':
                    turn_dict[t].append(turns[i+n]) 
                    n+=1
            except IndexError:
                # print(t)
                #NB: this is because it hits the end of the list
                continue
    return turn_dict

In [ ]:
jfile = dev_32

In [ ]:
labels = []
raw = []
input_text = []
MAX_LEN = 16

for game_ind, game in enumerate(jfile):

    # if game['id'] == 'C58-B1-A44':

    raw_text = [j["speaker"][:5] + ": " + j["text"] for j in jfile[game_ind]["edus"] ]
    raw += [raw_text]

    #make raw text 

    #get candidates
    game_cands = []
    edus = game['edus']
    turns = get_turns(edus)
    #dict of turns to avoid
    # print(turns)

    ##get edu list
    archs = [t for t in edus if t['speaker'] != 'Builder' and t['turn_ind'] <= 3]

    ##make relations list for easy access during candidate production.
    rels = [(r['x'], r['y']) for r in game['relations']]
    # print(rels)

    for edu in archs:
        num = edu['turn']
        avoid = []
        #then this will be source
        if edu['turn'] in turns.keys():
            avoid = turns[edu['turn']]
        for target in [elem for elem in archs if elem['turn'] > num and elem['turn'] not in avoid]:
            cand = [game_ind, edu['global_index'], target['global_index'], 0, -1, edu['res'], edu['res']]
            if (cand[1], cand[2]) in rels: ##NB need to figure out how to find rels that arent captured
                cand[3] = 1
                cand[4] = 14
            game_cands.append(cand)
    # print(game_cands)

    #reduce cands to those under specified cutoff
    if MAX_LEN:
        game_cands = [c for c in game_cands if abs(c[2] - c[1]) <= MAX_LEN]
    # for ca in game_cands:
    #     print(ca[2] - ca[1])

    #create input text pairs from candidates

    text = [[raw_text[cand[1]], raw_text[cand[2]]] for cand in game_cands]


    labels.extend(game_cands)
    input_text.extend(text)

    print('Game {} has length {} and  has {} candidates'.format(game['id'], len(raw_text), len(game_cands)))

# use this to see the distance distributions
# distances = [abs(i[2] - i[1]) for i in labels if i[3] == 1]
# dist_cnts = Counter(distances)
# cntslist = list(dist_cnts.items())
# cntslist.sort(key=lambda x:x[0])

# for c in cntslist:
#     print(c[0], [c[1]])


final_outputs = [raw, input_text, labels]

In [ ]:
len(final_outputs[1])

In [ ]:
len(final_outputs[2])

In [ ]:
Counter([i[6] for i in final_outputs[2]])

[[0, 0, 1, 1, 9, 1],
format for attachemnts : game, x, y, gold att, type, predicted attach

In [ ]:
attach_preds = home + '/bert_dev_attachments.pkl'

In [ ]:
fi = open(attach_preds, 'rb')
# dump information to that file
attach = pickle.load(fi)
# close the file
fi.close()

In [ ]:
dev_32[0].keys()

In [ ]:
len(attach)

In [ ]:
attach[0]

check F1 score

In [ ]:
correct = [i[3] for i in attach]
predicted = [i[5] for i in attach]

In [ ]:
precision_recall_fscore_support(correct, predicted, average='binary')

In [ ]:
Counter([i[3] for i in attach])

In [ ]:
Counter([i[5] for i in attach])

In [ ]:
Counter([i[5] for i in attach if i[4] == -1])

In [ ]:
Counter([reverse_map[i[4]] for i in attach])

In [ ]:
Counter([reverse_map[i[4]] for i in attach if i[5] == 1])

In [ ]:
#true positives
true_pos = [i for i in attach if i[5] == 1 and i[3] == 1]

In [ ]:
len(true_pos)

In [ ]:
#false positives
false_pos = [i for i in attach if i[5] == 1 and i[3] == 0]

In [ ]:
len(false_pos)

In [ ]:
#false negatives
false_neg = [i for i in attach if i[5] == 0 and i[3] == 1]

In [ ]:
len(false_neg)

In [ ]:
#true negatives
len([i for i in attach if i[5] == 0 and i[3] == 0])

In [ ]:
##ok now see how the relations pan out in false positives

Lengths of false positive predictions

In [ ]:
false_pos[0]

In [ ]:
#make false pos 
fp_lens = Counter([i[2]-i[1] for i in false_pos])

In [ ]:
for f in fp_lens.items():
    print('{} : {}'.format(f[0], f[1]))

In [ ]:
import pandas
import numpy 
import matplotlib.pyplot as plt

True positive counts

In [ ]:
tp_lens = [(i[4], i[2]-i[1]) for i in true_pos]

In [ ]:
tp_lens[:2]

In [ ]:
lens_dict = defaultdict(list)

In [ ]:
for f in tp_lens:
    lens_dict[f[0]].append(f[1])

In [ ]:
for d in lens_dict.keys():
    print(reverse_map[d])
    print(Counter(lens_dict[d]))
    print('------------------')

False Negatives counts

In [ ]:
fn_lens = [(i[4], i[2]-i[1]) for i in false_neg]

In [ ]:
fn_lens[:4]

In [ ]:
fn_lens_dict = defaultdict(list)

In [ ]:
for f in fn_lens:
    fn_lens_dict[f[0]].append(f[1])

In [ ]:
lens_dict[9]

In [ ]:
for d in fn_lens_dict.keys():
    print(reverse_map[d])
    print(Counter(fn_lens_dict[d]))
    print('------------------')

create json

In [ ]:
len(dev_32)

In [ ]:
attach[0]

In [ ]:
dev_32[0].keys()

In [ ]:
true_pos_dev = []
for i in range(32):
    new_relations = []
    for a in attach:
        if a[0] == i and a[5] == 1 and a[3] == 0:
            rel = {}
            rel['x'] = a[1]
            rel['y'] = a[2]
#             rel['type'] = reverse_map[a[4]]
            rel['type'] = 'false_positive'
            new_relations.append(rel)
    new_game = {}
    new_game['id'] = dev_32[i]['id']
    new_game['edus'] = dev_32[i]['edus']
    new_game['relations'] = new_relations
    true_pos_dev.append(new_game)


In [ ]:
len(true_pos_dev)

In [ ]:
for t in true_pos_dev:
    print(len(t['relations']))

In [ ]:
#save the json 
save_path = home + '/DEV_32_bert_attach_fp.json'
with open(save_path, "w") as outfile:
    json.dump(true_pos_dev, outfile)

# last function

In [ ]:
len(dev_32)

In [ ]:
len(dev_32[0]['edus'])

In [ ]:
cutoff = 10
flat_last = []
flat_gold = []
for game in dev_32:
    #make last
    last = [(i, i+1) for i in range(len(game['edus']) - 1)]
    #get all rels
    rels = [(r['x'], r['y']) for r in game['relations'] if (r['y'] > r['x'] and (r['y'] - r['x']) <= cutoff)]
    #merge lists
    for l in last:
        flat_last.append(1)
        if l in rels:
            flat_gold.append(1)
        else:
            flat_gold.append(0)
    for r in rels:
        if r not in last:
            flat_gold.append(1)
            flat_last.append(0)

In [ ]:
len(flat_gold), len(flat_last)

In [ ]:
scores = precision_recall_fscore_support(flat_gold, flat_last, average='binary')

In [ ]:
scores